In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import joblib
from scipy import stats

In [2]:
pip install gradio

     |████████████████████████████████| 2.9 MB 6.6 MB/s 
     |████████████████████████████████| 253 kB 50.2 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 2.0 MB 45.6 MB/s 
     |████████████████████████████████| 211 kB 67.6 MB/s 
     |████████████████████████████████| 1.1 MB 11.0 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 53.7 MB/s 
     |████████████████████████████████| 271 kB 57.5 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 10.9 MB 40.8 MB/s 
     |████████████████████████████████| 58 kB 3.9 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 3.6 MB 47.8 MB/s 
     |████████████████████████████████| 856 kB 53.8 MB/s 
     |██████████████████████

In [10]:
import gradio as gr

In [23]:
copy_data = pd.read_csv('/content/ibm_dataset.csv')
X_train = pd.read_csv('/content/X_train.csv', index_col= 0)
X_train.head()

,binarized_Gender,binarized_OverTime,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,MaritalStatus_Divorced,MaritalStatus_Married,...,PercentSalaryHike,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsWithCurrManager,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,RelationshipSatisfaction,WorkLifeBalance
1194,0,0,0,0,1,0,0,1,1,0,...,-0.337129,-0.605389,-0.665706,-0.625365,-0.616406,2,4,2,3,3
128,1,0,0,0,1,0,1,0,0,1,...,-0.337129,-0.605389,-0.830071,-0.905635,-0.897047,3,3,4,3,3
810,1,0,0,0,1,0,0,1,0,1,...,-0.880974,0.190962,0.813578,1.336527,1.348076,1,3,3,4,3
478,1,0,0,0,1,0,0,1,0,1,...,-1.152896,-1.401740,-0.008246,-0.064824,0.506155,2,3,3,3,3
491,1,1,0,1,0,0,1,0,1,0,...,-0.609051,0.190962,0.156119,0.775986,0.786795,4,3,3,2,3


In [24]:
svm_clf2 = joblib.load('/content/svm2.pkl')

In [13]:
test_input = np.zeros( (1, 24))

def attrition_prob(gender, ot, travel, department, m_stat, age, rate, distance):
    test_input = np.zeros( (1, 24))
    if gender == "Male":
        test_input[:, 0 ] =1
    else:
        test_input[:, 0 ] =0


    if ot == "Yes":
        test_input[:, 1 ] =1
    else:
        test_input[:, 1 ] =0


    if travel == "None":
        test_input[:, 2 ] =1
    elif travel =='Rarely':
        test_input[:, 4 ] =1
    elif travel =='Frequently':
        test_input[:, 3 ] =1


    if department == "HR":
        test_input[:, 5 ] =1
    elif department =='R&D':
        test_input[:, 6 ] =1
    elif department =='Sales':
        test_input[:, 7 ] =1



    if m_stat == "Divorced":
        test_input[:, 8 ] =1
    elif m_stat =='Married':
        test_input[:, 9 ] =1
    elif m_stat =='Single':
        test_input[:, 10 ] =1

    test_input[:, 11 ] = (age - copy_data['Age'].mean()) / copy_data['Age'].std()
    
    test_input[:, 12 ] = (rate - copy_data['MonthlyRate'].mean()) / copy_data['MonthlyRate'].std()
    test_input[:, 13 ] = (distance - copy_data['DistanceFromHome'].mean()) / copy_data['DistanceFromHome'].std()

    test_input[:, 14: ]  = X_train.iloc[:, 14:].mean(axis = 0)

    pred = svm_clf2.predict_proba(test_input)
    leaving_prob = round(float(pred[:, -1]), 4) * 100
    prob_pred = np.max(svm_clf2.predict_proba(test_input))
    predicted_class_idx = np.argmax(svm_clf2.predict_proba(test_input))
    if predicted_class_idx == 0:
        predicted_class = 'Stay'
    else:
        predicted_class = 'Leave'

    
    return {"Stay": float(pred[:, 0]), "Leave": float(pred[:, 1])}
    
    
    # test_input, pred, prob_pred, predicted_class, f'The probability of leaving: {leaving_prob}%'

In [27]:
attrition_prob('Male', 'Yes', 'Rarely', 'R&D', 'Married', 50, 30000, 30)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"


{'Leave': 0.15034190194109043, 'Stay': 0.8496580980589096}

In [28]:
input1 = gr.inputs.Dropdown(['Female', 'Male'], label="Gender")
input2 = gr.inputs.Dropdown(['Yes', 'No'], label="Is this role required to work overtime?", default = 'Yes')
input3 = gr.inputs.Radio(choices = ['None', 'Rarely', 'Frequently'], label="Travel frequency of this applied role")
input4 = gr.inputs.Radio(choices = ['HR', 'R&D', 'Sales'], label="Applied Department")
input5 = gr.inputs.Radio(choices = ['Single', 'Married', 'Divorced'], label="Marital Status")
input6 = gr.inputs.Slider(minimum=20, maximum=65, step=1, default=35, label= 'Age', optional=False)
input7 = gr.inputs.Number(label = 'Expected Salary (THB)')
input8 = gr.inputs.Number(label = 'Distance from Home (Km)')

# input2 = gr.inputs.Textbox(placeholder="Enter Last Name")

output = gr.outputs.Label(num_top_classes=2, label = 'Probabiliy distribution') 

gui = gr.Interface(fn=attrition_prob,
                   inputs= [input1, input2, input3, input4, input5, input6, input7, input8],
                   outputs=[output],
                   title="Predicted attrition rate of new hires",
                   description = 'This system helps determine the likelihood of IBM new hires leavning the organization',
                   theme = 'dark-grass')

gui.launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://45055.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7fa094ce6f50>,
 'http://127.0.0.1:7860/',
 'https://45055.gradio.app')